In [ ]:
class PykrxDataLoader:
    def __init__(self, fromdate:str, todate: str, market: str = "KOSPI"):
        self.fromdate = fromdate
        self.todate = todate
        self.market = market
        
    # 주가 데이터 불러오기
    def load_stock_data(self, ticker_list: List, freq: str, delay: float = 1):
        ticker_data_list = []
        for ticker in ticker_list:
            ticker_data = stock.get_market_ohlcv(fromdate=self.fromdate,
                                                 todate=self.todate,
                                                 ticker=ticker,
                                                 freq='d',
                                                 adjusted=True)
            ticker_data = ticker_data.rename(
                columns={'시가' : 'open', '고가' : 'high', '저가' : 'low',
                         '종가' : 'close', '거래량' : 'volume',
                         '거래 대금' : 'trading_value', '등락률': 'change_pct'}
            )
            ticker_data = ticker_data.assign(ticker=ticker)
            ticker_data.index.name = 'date'
            ticker_data_list.append(ticker_data)
            time.sleep(delay)
        data = pd.concat(ticker_data_list)
        # 잠시 거래를 중단한 주가의 시가 ,고가, 저가 보충
        data.loc[data.open == 0,
                 ['open', 'high', 'low']] = data.loc[data.open == 0 , 'close']
        # 샘플링을 통해 일 데이터를 다른 주기 데이터로 변환
        if freq != 'd':
            rule = {
                'open' : 'first',
                'high' : 'max',
                'low' : 'min',
                'close' : 'last',
                'volume' : 'sum',
                
            }